In [ ]:
import ollama
import requests
import json

In [ ]:

response = ollama.chat(model='llama2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

In [11]:
def generate_workflow(name, description, initiateUsing, ocr, processNumberPrefix, linkMasterFormId, smtpEmailId,
                      coordinator=[], superuser=[], slaSettings={"workDays": [], "workHours": {"from": "", "to": ""}, "timeZone": "",
                                                               "slaType": "STANDARD", "slaMasterFormId": 0, "workflowFormSLAFieldId": "", 
                                                               "emailNotify": True, "smsNotify": False},
                      kanbanSettings=[], scheduleReport={"durationType": "Week", "duration": "", "time": "", "notifyTo": [],
                                                         "runningRequest": False, "completedRequest": False},
                      publishOption="PUBLISHED", publishSchedule="", unpublishSchedule="", blocks=[]):
    workflow_blocks = []
    for block in blocks:
        action = "Reject" if block["proceedAction"] == "Reject" else "Approve"
        remarks = "true" if "remarks" in block.keys() and bool(block["remarks"]) else "false"
        passwordAccess = "false" if "passwordAccess" not in block.keys() else str(bool(block["passwordAccess"]))
        confirm = "false" if "confirm" not in block.keys() else str(bool(block["confirm"]))
        workflow_blocks.append({"id": block["id"], "left": block["left"], "top": block["top"], "fromBlockId": block["fromBlockId"],
                                "toBlockId": block["toBlockId"], "proceedAction": action, "remarks": remarks,
                                "passwordAccess": passwordAccess, "confirm": confirm})

    workflow = {
        "name": name,
        "description": description,
        "initiateUsing": initiateUsing,
        "ocr": ocr,
        "processNumberPrefix": processNumberPrefix,
        "linkMasterFormId": linkMasterFormId,
        "smtpEmailId": smtpEmailId,
        "coordinator": coordinator,
        "superuser": superuser,
        "slaSettings": slaSettings,
        "kanbanSettings": kanbanSettings,
        "scheduleReport": scheduleReport,
        "publish": {"publishOption": publishOption, "publishSchedule": publishSchedule, "unpublishSchedule": unpublishSchedule},
        "blockStatus": 0,
        "mlPredictions": [],
        "hasSLASettings": bool(slaSettings),
        "initiateUserDomain": []
    }

    if workflow_blocks:
        workflow["blockStatus"] = 1
        workflow["modifiedBlockIds"] = []
        workflow["workflowBlocks"] = workflow_blocks

    return json.dumps(workflow)

def get_workflow_arguments(workflow_type):
    prompt = f"""Generate arguments for a {workflow_type} workflow in JSON format with the following structure:
    {{
        "name": "Workflow name",
        "description": "Detailed workflow description",
        "initiateUsing": {{"type": "FORM", "repositoryId": 0, "formId": <number>}},
        "ocr": boolean,
        "processNumberPrefix": [prefix configuration],
        "linkMasterFormId": <number>,
        "smtpEmailId": <number>,
        "coordinator": [],
        "superuser": [],
        "blocks": [{{
            "id": "string",
            "left": number,
            "top": number,
            "fromBlockId": "string",
            "toBlockId": "string",
            "proceedAction": "Approve/Reject",
            "remarks": boolean,
            "passwordAccess": boolean,
            "confirm": boolean
        }}]
    }}
    Return only valid JSON without any additional text or explanations."""

    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])
    
    result = response['message']['content']
    result = result.strip()
    
    # Clean the response to ensure valid JSON
    if result.startswith("```json"):
        result = result.split("```json")[1]
    if result.endswith("```"):
        result = result.rsplit("```", 1)[0]
    
    result = result.strip()

    
    try:
        return json.loads(result)
    except json.JSONDecodeError:
        # Provide a default workflow structure if JSON parsing fails
        return {
            "name": f"{workflow_type} Workflow",
            "description": f"Standard {workflow_type} process workflow",
            "initiateUsing": {"type": "FORM", "repositoryId": 0, "formId": 1},
            "ocr": False,
            "processNumberPrefix": ["WF"],
            "linkMasterFormId": 1,
            "smtpEmailId": 1,
            "coordinator": [],
            "superuser": [],
            "blocks": [
                {
                    "id": "start",
                    "left": 100,
                    "top": 100,
                    "fromBlockId": "start",
                    "toBlockId": "end",
                    "proceedAction": "Approve",
                    "remarks": True,
                    "passwordAccess": False,
                    "confirm": True
                }
            ]
        }

def create_workflow_from_type(workflow_type):
    args = get_workflow_arguments(workflow_type)
    return generate_workflow(
        name=args["name"],
        description=args["description"],
        initiateUsing=args["initiateUsing"],
        ocr=args["ocr"],
        processNumberPrefix=args["processNumberPrefix"],
        linkMasterFormId=args["linkMasterFormId"],
        smtpEmailId=args["smtpEmailId"],
        coordinator=args["coordinator"],
        superuser=args["superuser"],
        blocks=args["blocks"]
    )

def get_user_workflow():
    workflow_type = input("\nEnter the type of workflow you want to generate: ")
    return workflow_type

def generate_workflow_from_user_input():
    workflow_type = get_user_workflow()
    workflow_json = create_workflow_from_type(workflow_type)
    filename = f"{workflow_type.lower().replace(' ', '_')}_workflow.json"
    with open(filename, 'w') as f:
        f.write(workflow_json)
        
    print(f"\nWorkflow generated successfully and saved to {filename}")
    return workflow_json

if __name__ == "__main__":
    generated_workflow = generate_workflow_from_user_input()



Workflow generated successfully and saved to expense_reimbursement_process_workflow.json
